# Análise exploratória de dados - GamersClub CS:GO

In [425]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
plt.style.use('seaborn')


C:\Users\danie\AppData\Local\Temp\ipykernel_11624\2276442144.py:11: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.



## Introdução

O jogo Counter-Strike Global Offensive, mais conhecido como CSGO, é um jogo de tiro em primeira pessoa (First Person Shooter), em que partidas são realizadas entre equipes de 5 jogadores cada. O jogo acaba quando uma das equipes ganha 16 rodadas. No caso de empate (por um placar de 15x15), são jogadas prorrogações com 6 rodadas até que uma das equipes ganhe 4 de 6 rodadas. O jogo compõe parte importante do cenário de esporte eletrônico, sendo acompanhado diariamente por milhares de espectadores em todos os países do mundo, apresentando cada vez mais espectadores no Brasil e atraindo uma quantidade considerável de investimentos.

A *GamersClub* é uma plataforma digital brasileira que tem como objetivo prover o serviço de criação de partidas para o CSGO, além de promover competições profissionais e aulas para aqueles que pagam pela assinatura do serviço.

O *dataset* utilizado nesse trabalho foi disponibilizado pela própria plataforma da *GamersClub* a partir do site do *Kaggle*. Esse conjunto de dados apresenta as informações de partidas jogadas por membros da plataforma, contendo informações de mais de 150 mil partidas. O mesmo pode ser obtido a partir do seguinte [endereço](https://www.kaggle.com/datasets/gamersclub/brazilian-csgo-plataform-dataset-by-gamers-club)

Os dados são apresentados de forma que cada linha represente uma partida de um determinado jogador. Dessa forma, todas as análises são feitas a nível individual, ou seja, não são analisados os dados de equipes, mas de jogadores. O conjunto de dados em si representa uma amostragem aleatória da plataforma. Nesse caso, a população analisada é o conjunto de **todos** os jogadores que participam da plataforma.

### Glossário

Utilitárias: elementos táticos do jogo, como:
- Granadas de fumaça, utilizadas para impedir a visão do time adversário
- Granada de fragmentação, utilizada para causar dano direto ao adversário através de explosão, sendo esse dano inversamente proporcional à distância da explosão para o jogador
- Granada incendiária, utilizada para impedir que jogadores adversários avancem sobre uma determinada região do mapa

ADR (*Average Damage per Round*): dano médio por rodada. Isto é, quanto um determinado jogador causou de dano na equipe inimiga a cada rodada, em média. Um jogador pode causar dano em um adversário a partir do uso de armas ou utilitárias (granadas).

HS (*Headshot*): um tiro que atinge a cabeça de um adversário. Estatística importante, pois é uma das maneiras mais eficientes de causar dano no adversário. À depender do armamento utilizado, causa uma eliminação instantânea.

## Parte 0: qualidade dos dados

In [426]:
lobby_players = pd.read_csv('data/tb_lobby_stats_player.csv')

In [427]:
lobby_players.head(10)

,idLobbyGame,idPlayer,idRoom,qtKill,qtAssist,qtDeath,qtHs,qtBombeDefuse,qtBombePlant,qtTk,...,qtFlashAssist,qtHitHeadshot,qtHitChest,qtHitStomach,qtHitLeftAtm,qtHitRightArm,qtHitLeftLeg,qtHitRightLeg,flWinner,dtCreatedAt
0,1,1,1,5,1,16,2,0,0,0.0,...,0.0,3.0,13.0,4.0,2.0,2.0,1.0,0.0,0,2022-01-21 19:45:44
1,2,1,2,24,3,18,6,0,4,0.0,...,0.0,7.0,26.0,14.0,2.0,1.0,1.0,3.0,1,2022-02-04 02:09:47
2,3,2,3,6,4,23,2,0,1,0.0,...,0.0,3.0,15.0,8.0,1.0,2.0,0.0,2.0,0,2021-09-18 18:07:43
3,3,391,27508,10,5,20,4,1,0,0.0,...,0.0,6.0,27.0,10.0,1.0,7.0,6.0,6.0,1,2021-09-18 18:07:43
4,4,2,4,8,4,26,6,0,2,0.0,...,2.0,8.0,19.0,12.0,2.0,3.0,2.0,5.0,0,2021-09-27 00:17:45
5,5,2,5,10,1,11,5,0,3,0.0,...,1.0,5.0,6.0,8.0,0.0,0.0,3.0,0.0,1,2021-09-29 22:05:47
6,5,1068,69976,9,2,19,4,1,0,0.0,...,1.0,6.0,21.0,14.0,3.0,4.0,2.0,2.0,0,2021-09-29 22:05:47
7,6,2,6,16,1,23,9,0,1,0.0,...,1.0,9.0,21.0,19.0,3.0,4.0,2.0,3.0,0,2021-10-07 22:48:43
8,7,2,7,11,2,19,2,0,1,0.0,...,1.0,5.0,17.0,12.0,0.0,4.0,0.0,2.0,0,2021-10-08 23:29:57
9,8,2,8,8,6,13,2,0,0,0.0,...,0.0,3.0,9.0,17.0,2.0,5.0,7.0,8.0,0,2021-10-12 16:48:09


In [428]:
# count lines
print('Number of lines: ', len(lobby_players))
# check if all (idLobbyGame, idPlayer) are unique
print('Number of unique (idLobbyGame, idPlayer) pairs: ', len(lobby_players.groupby(['idLobbyGame', 'idPlayer']).size()))
# check if all (idRoom, idPlayer) are unique
print('Number of unique (idRoom, idPlayer) pairs: ', len(lobby_players.groupby(['idRoom', 'idPlayer']).size()))
# USAR APENAS O LOBBYGAME

Number of lines:  184152
Number of unique (idLobbyGame, idPlayer) pairs:  184152
Number of unique (idRoom, idPlayer) pairs:  184152


In [429]:
lobby_players['idLobbyGame'].value_counts().sort_values(ascending=False).head(10)

58891    5
60420    4
17782    4
17622    4
71305    4
93223    4
17777    4
4508     4
17644    3
31686    3
Name: idLobbyGame, dtype: int64

In [430]:
lobby_players['idRoom'].value_counts().sort_values(ascending=False).head(10)

17861    3
17801    3
17818    3
17817    3
17858    3
17876    3
58765    3
24323    3
17887    3
17889    3
Name: idRoom, dtype: int64

Conforme os dados acima, nota-se que não será possível realizar uma análise por equipes, uma vez que partidas com múltiplos jogadores simultâneas são escassas. Desta forma, a análise proposta tem como alvo principal o jogador individualmente, explorando informações como mapas, nível na GC, quantidade de eliminações, mortes e assistências, entre outras.

## Parte 1: players

In [431]:
players = pd.read_csv('data/tb_players.csv')

In [432]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716 entries, 0 to 2715
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idPlayer        2716 non-null   int64 
 1   flFacebook      2716 non-null   int64 
 2   flTwitter       2716 non-null   int64 
 3   flTwitch        2716 non-null   int64 
 4   descCountry     2716 non-null   object
 5   dtBirth         2008 non-null   object
 6   dtRegistration  2716 non-null   object
dtypes: int64(4), object(3)
memory usage: 148.7+ KB


In [433]:
players.head(10)

,idPlayer,flFacebook,flTwitter,flTwitch,descCountry,dtBirth,dtRegistration
0,1,0,0,0,cl,NaN,2021-03-19 21:31:39
1,2,0,0,0,br,NaN,2020-05-06 19:28:29
2,3,0,0,0,br,NaN,2020-01-28 13:00:38
3,4,0,0,0,br,NaN,2017-05-31 16:13:51
4,5,0,0,0,ar,NaN,2021-02-08 11:57:21
5,6,0,0,0,br,2002-03-23 00:00:00,2019-01-09 19:24:27
6,7,0,0,0,br,NaN,2020-04-24 22:53:50
7,8,0,0,0,br,NaN,2021-06-26 14:47:57
8,9,0,0,0,ar,2000-05-03 00:00:00,2016-05-22 15:12:48
9,10,0,0,0,br,2005-04-15 00:00:00,2018-02-28 11:29:11


In [434]:
print(players['flFacebook'].value_counts())
print(players['flTwitter'].value_counts())
print(players['flTwitch'].value_counts())

0    2653
1      63
Name: flFacebook, dtype: int64
0    2672
1      44
Name: flTwitter, dtype: int64
0    2663
1      53
Name: flTwitch, dtype: int64


Explicação das colunas:
* ``idPlayer``: id do player
* ``flFacebook``, ``flTwitter``, ``flTwitch``: se o jogador sincronizou (1) ou não (0) sua conta com cada uma das plataformas
* ``descCountry``: sigla de dois dígitos indicando o país do jogador
* ``dtRegistration``: data de ingresso do jogador na plataforma GamersClub

In [435]:
players['descCountry'].isna().sum()

0

Ou seja, todos os players tem um país.

In [436]:
countries_count = players['descCountry'].value_counts()
freq_countries = countries_count[countries_count > 100]
freq_countries

br    2071
ar     491
Name: descCountry, dtype: int64

In [437]:
# Plot to donut chart
fig = px.pie(countries_count, values=countries_count.values, names=countries_count.index, title='Países')
fig.update_traces(textposition='inside', textinfo='percent+label')
# # add hole
fig.update_traces(hole=.6, hoverinfo="label+percent+name")
fig.show()


In [438]:
# # Calculate the age of the players
players['age'] = (dt.datetime.now() - pd.to_datetime(players['dtBirth'])).astype('<m8[Y]')

In [439]:
# Histogram of player age
# Distribution curve over histogram
# Preparing data
x = np.array(players['age'])
# Remove NaN values
x = x[~np.isnan(x)]
# Remove x bigger than 47
x = x[x < 47]
print(x)
# Limit n bins to 20
fig = ff.create_distplot([x], ['Idade dos jogadores'], bin_size=1, show_rug=False, histnorm='probability', curve_type='normal')
# y = Densidade
# x = Idade
fig.update_layout(
    xaxis_title='Idade',
    yaxis_title='Densidade',
)
# fig width = 400px
fig.update_layout(width=800)
# Graph title = "Distribuição de idade dos jogadores da plataforma Gamers Club"
fig.update_layout(title='Distribuição de idade dos jogadores da plataforma Gamers Club')
# Set curve color to black
fig.update_traces(marker_color='#393E46')
# Set graph color to orange
fig.update_traces(selector=dict(type='histogram'), marker_color='orange')
fig.show()

[21. 23. 18. ... 23. 22. 23.]


**Nota: barras do histograma limitadas. Isso causou a remoção dos outliers: jogadores mais velhos que 45 anos. Haviam apenas 8.**

In [440]:
# boxplot
fig = px.box(players, y='age', title='Boxplot da idade dos jogadores da plataforma Gamers Club')
fig.update_layout(yaxis_title='Idade')
fig.show()

In [441]:
# mean
player_age_mean = players['age'].mean()
print('Média de idade dos jogadores: ', player_age_mean)

Média de idade dos jogadores:  24.782868525896415


In [442]:
# classify by skewness = mean - mode
player_age_skewness = player_age_mean - players['age'].mode()[0]
player_age_skewness

1.782868525896415

**Assimétrica à direita**

In [443]:
# coeficiente de assimetria de Pearson
player_age_skew_coeff = 3 * (players['age'].mean() - players['age'].median()) / players['age'].std()
player_age_skew_coeff

0.9674230345690763

**Assimetria moderada**

In [444]:
player_age = np.array(players['age'])
# remove nan
player_age = player_age[~np.isnan(player_age)]
q3 = np.quantile(player_age, 0.75)
q1 = np.quantile(player_age, 0.25)
p90 = np.quantile(player_age, 0.90)
p10 = np.quantile(player_age, 0.10)
player_age_kurtosis = (q3 - q1) / (2 * (p90 - p10))
player_age_kurtosis

0.2692307692307692

**Platiocúrtica (ligeiramente)**

In [445]:
# How mah players in total
print(players['age'].count())
# How many players older than the mean age
print(players[players['age'] > player_age_mean]['age'].count())
# Percentage of players older than the mean age
print(players[players['age'] > player_age_mean]['age'].count() / players['age'].count())

2008
866
0.4312749003984064


Aproximadamente 43% dos players tem idade acima da média, o que explica a assimetria moderada à direita.

## Parte 2: resto das coisas

falar que os dados são tendenciosos

necessária análise por nível

In [446]:
# group by idPlayer, get the max vlLevel that the player reached
player_levels = lobby_players.groupby('idPlayer')['vlLevel'].max()
# to np array
player_level = np.array(player_levels)

### Distribuição de level

In [447]:
# Histogram of player level
fig = px.histogram(player_levels, x='vlLevel', title='Histograma do nível dos jogadores da plataforma Gamers Club')
fig.update_layout(xaxis_title='Nível')
fig.show()

In [448]:
# Histogram of player level
# Distribution curve over histogram
# Preparing data
x = np.array(player_levels)
print(x)
# Limit n bins to 20
fig = ff.create_distplot([x], ['Nível dos jogadores'], bin_size=1, show_rug=False, histnorm='probability', curve_type='normal')
# y = Densidade
# x = Idade
fig.update_layout(
    xaxis_title='Nível',
    yaxis_title='Densidade',
)
# fig width = 400px
fig.update_layout(width=800)
# Graph title = "Distribuição de idade dos jogadores da plataforma Gamers Club"
fig.update_layout(title='Distribuição de nível dos jogadores da plataforma Gamers Club')
# Set curve color to black
fig.update_traces(marker_color='#393E46')
# Set graph color to orange
fig.update_traces(selector=dict(type='histogram'), marker_color='orange')
fig.show()

[10  3 15 ... 12  7  9]


In [449]:
# boxplot
fig = px.box(lobby_players, y='vlLevel', title='Boxplot do nível dos jogadores da plataforma Gamers Club')
fig.update_layout(yaxis_title='Nível')
fig.show()

In [450]:
# mean
player_level_mean = lobby_players['vlLevel'].mean()
print('Média de nível dos jogadores: ', player_level_mean)

Média de nível dos jogadores:  13.668350058647205


In [451]:
# get mode
player_level_mode = lobby_players['vlLevel'].mode()[0]
player_level_mode 

20

In [452]:
# classify by skewness
# skew calc: mean - mode
skew_calc = player_level_mean - player_level_mode
skew_calc

-6.331649941352795

Assimétrica à esquerda

In [453]:
player_level_skew_coeff = 3 * (lobby_players['vlLevel'].mean() - lobby_players['vlLevel'].median()) / lobby_players['vlLevel'].std()
abs(player_level_skew_coeff)

0.21237613346875273

Assimetria moderada

In [454]:
# kurtosis (q3 - q1) / 2(p90 - p10)
q3 = lobby_players['vlLevel'].quantile(0.75)
q1 = lobby_players['vlLevel'].quantile(0.25)
p90 = lobby_players['vlLevel'].quantile(0.90)
p10 = lobby_players['vlLevel'].quantile(0.10)
kurtosis_calc = (q3 - q1) / (2 * (p90 - p10))
kurtosis_calc

0.3076923076923077

Platicúrtica

Percebe-se uma distribuição mais acentuada nos níveis mais altos da plataforma, com a moda sendo o nível máximo, 20.

Isto pode ser explicado pelo fato da plataforma direcionar-se a jogadores mais avançados, que jogam profissionalmente ou como hobby "sério". 
Um dos diferenciais da plataforma GamersClub é oferecer melhores servidores do que os tradicionais do jogo, além de possibilitar montar times, participar de campeonatos, dentre outras funcionalidades que atraem, justamente, jogadores mais empenhados.

Jogadores mais casuais tendem a usar o sistema de partidas padrão do jogo.

[explicar melhor que a GC é "independente" no começo]

### Mapas mais jogados

In [455]:
maps = lobby_players['descMapName'].value_counts()
# Plot em gráfico de barras
fig = px.bar(maps, x=maps.index, y=maps.values, title='Mapas')
fig.update_layout(xaxis_title='Mapa', yaxis_title='Quantidade de partidas')
fig.show()

In [456]:
# find out most recent date by column dtCreatedAt
print(lobby_players['dtCreatedAt'].max())
# find out oldest date by column dtCreatedAt
print(lobby_players['dtCreatedAt'].min())
# how many in 2021
print(lobby_players[lobby_players['dtCreatedAt'] >= '2021-01-01']['dtCreatedAt'].count())
# how many in 2022
print(lobby_players[lobby_players['dtCreatedAt'] >= '2022-01-01']['dtCreatedAt'].count())

2022-02-11 13:27:48
2021-09-14 13:06:52
184152
63334


Esta distribuição é explicada pelo fato da comunidade de Counter-Strike ser notadamene conservadora, isto é, resistente À mudanças.

Acient e Overpass são mapas exclusivos à última versão do jogo, Global Offensive.
Deste modo, houve uma resistência inicial muito grande da comunidade à estes mapas, que perdura até hoje, explicando o fato de não serem muito jogados.

Mirage, Inferno e Dust II, por outro lado, são mapas populares desde a primeira versão de Counter-Strike.

Vertigo, mesmo disponível desde a primeira versão de Counter-Strike, não se tornou popular até 2019, quando o mapa foi retrabalhado.

Train costumava ser um mapa popular, mas foi abandonado pelos campeonatos profissionais, não sendo mais atualizado e, portanto, tendo sua popularidade decaindo com o tempo.

Nuke é um mapa conhecidamente desequilibrado ([há vantagem para o lado Contraterrosita](https://draft5.gg/noticia/inferno-e-o-mapa-mais-balanceado-da-rotacao-nuke-e-onde-os-cts-tem-mais-vantagem)), sendo mais jogado em partidas profissionais ou amadoras sérias.
Este último fato levou o grupo a investigar a correlação entre mapas e os níveis dos jogadores.

### Mapas x níveis

In [457]:
# group by descMapName, levels and count
maps_levels = lobby_players.groupby(['descMapName', 'vlLevel']).size().reset_index(name='count')
maps_levels

,descMapName,vlLevel,count
0,de_ancient,0,8
1,de_ancient,1,2
2,de_ancient,2,9
3,de_ancient,3,39
4,de_ancient,4,44
...,...,...,...
171,de_vertigo,17,1378
172,de_vertigo,18,1599
173,de_vertigo,19,1861
174,de_vertigo,20,2299


In [458]:
# grouped bar chart
# y = count
# group by descMapName
# x = vlLevel
fig = px.bar(maps_levels, x='vlLevel', y='count', color='descMapName', title='Mapas por nível')
fig.update_layout(xaxis_title='Nível', yaxis_title='Quantidade de partidas')
fig.show()